# Clebsch-Gordan Coefficients for SU(2)

## Preamble

In [1]:
import sympy as sp
from sympy.physics.quantum import TensorProduct

## Generators of the Irreducible Representations of SU(2)

In [2]:
def Jp(j0):
  # $J^{+}$
  j=sp.nsimplify(j0,rational=True)
  res=sp.zeros(2*j+1)
  for i in range(2*j+1):
    m=j-i
    res[i-1,i]=sp.sqrt((j+m+1)*(j-m))
  return res

def Jm(j0):
  # $J^{-}$
  return sp.conjugate(sp.transpose(Jp(j0)))

def J(a,j0):
  # $J_i$
  if a==3:
    j=sp.nsimplify(j0,rational=True)
    res=sp.zeros(2*j+1)
    for i in range(2*j+1):
      m=j-i
      res[i,i]=m
    return res
  elif a==1:
    return (Jp(j0)+Jm(j0))/2
  elif a==2:
    return -sp.I*(Jp(j0)-Jm(j0))/2

def JJ(j):
  # Quadratic Casimir $J_iJ_i$
  return (J(1,j)**2+J(2,j)**2+J(3,j)**2).simplify()

## Eigenstates

In [3]:
def s(j,m):
  # Spin State |j,m>
  d=int(2*j+1)
  i=sp.nsimplify(j-m,rational=True)
  return sp.eye(d).col(i)

## Direct Product Generators

In [4]:
def DJ(a,j01,j02):
  # Total Spin, 'a' Specifies the Component, j1 and j2 the Spins of the Individual Particles
  j1=sp.nsimplify(j01,rational=True)
  j2=sp.nsimplify(j02,rational=True)
  return TensorProduct(sp.Matrix(J(a,j1)),sp.eye(2*j2+1))+TensorProduct(sp.eye(2*j1+1),sp.Matrix(J(a,j2)))

## Quadratic Casimir $|\mathbf{J}|^2$ in the Direct Product Basis

In [5]:
def DJJ(j1,j2):
  return sp.simplify(DJ(1,j1,j2)**2+DJ(2,j1,j2)**2+DJ(3,j1,j2)**2)

## Direct Product Basis

In [6]:
def Ds(j1,m1,j2,m2):
  # States |j1,m1>|j2,m2>
  return TensorProduct(sp.Matrix(s(j1,m1)),sp.Matrix(s(j2,m2)))

## Modal Matrix

In [7]:
def modal(j01,j02):
  """
  This Function Normalizes the Eigenvectors in the Columns of the Modal Matrix and
  Rephases it According to the Condon-Shortley Convention
  And Returns the Modal Matrix and the Diagonalized Quadratic Casimir
  """
  j1=sp.nsimplify(j01,rational=True)
  j2=sp.nsimplify(j02,rational=True)
  O0,JJD=DJJ(j1,j2).diagonalize()
  O1=sp.simplify(O0*sp.sqrt(sp.transpose(O0)*O0)**(-1))
  k=(2*j1+1)*(2*j2+1)
  sig=sp.eye(k)
  for b in range(k):
    for a in range(k):
      if O1[a,b]!=0:
        sig[b,b]=sp.sign(O1[a,b])
        break
  return O1*sig, JJD

## Table of Clebsch-Gordan Coefficients

In [12]:
def table_cg(j01,j02):
  j1=sp.nsimplify(j01,rational=True)
  j2=sp.nsimplify(j02,rational=True)
  modal_matrix,JJ_diag=modal(j1,j2)
  J3_diag=sp.transpose(modal_matrix)*DJ(3,j1,j2)*modal_matrix
  dim=(DJJ(j1,j2).shape)[0]
  for i in range(dim):
    J=(-1+sp.sqrt(1+4*JJ_diag[i,i]))/2
    M=J3_diag[i,i]
    for i1 in range(2*j1+1):
      m1=-j1+i1
      for i2 in range(2*j2+1):
        m2=-j2+i2
        if m1+m2==M:
          res=(sp.transpose(Ds(j1,m1,j2,m2))*modal_matrix.col(i))[0]
          if res!=0:
            print(f'<{j1},{m1};{j2},{m2}|{J},{M}> = {res}')

In [13]:
table_cg(1,1)

<1,-1;1,1|0,0> = sqrt(3)/3
<1,0;1,0|0,0> = -sqrt(3)/3
<1,1;1,-1|0,0> = sqrt(3)/3
<1,0;1,1|1,1> = -sqrt(2)/2
<1,1;1,0|1,1> = sqrt(2)/2
<1,-1;1,1|1,0> = -sqrt(2)/2
<1,1;1,-1|1,0> = sqrt(2)/2
<1,-1;1,0|1,-1> = -sqrt(2)/2
<1,0;1,-1|1,-1> = sqrt(2)/2
<1,1;1,1|2,2> = 1
<1,0;1,1|2,1> = sqrt(2)/2
<1,1;1,0|2,1> = sqrt(2)/2
<1,-1;1,1|2,0> = sqrt(6)/6
<1,0;1,0|2,0> = sqrt(6)/3
<1,1;1,-1|2,0> = sqrt(6)/6
<1,-1;1,0|2,-1> = sqrt(2)/2
<1,0;1,-1|2,-1> = sqrt(2)/2
<1,-1;1,-1|2,-2> = 1


In [14]:
table_cg(2,2)

<2,-2;2,2|0,0> = sqrt(5)/5
<2,-1;2,1|0,0> = -sqrt(5)/5
<2,0;2,0|0,0> = sqrt(5)/5
<2,1;2,-1|0,0> = -sqrt(5)/5
<2,2;2,-2|0,0> = sqrt(5)/5
<2,-1;2,2|1,1> = -sqrt(5)/5
<2,0;2,1|1,1> = sqrt(30)/10
<2,1;2,0|1,1> = -sqrt(30)/10
<2,2;2,-1|1,1> = sqrt(5)/5
<2,-2;2,2|1,0> = -sqrt(10)/5
<2,-1;2,1|1,0> = sqrt(10)/10
<2,1;2,-1|1,0> = -sqrt(10)/10
<2,2;2,-2|1,0> = sqrt(10)/5
<2,-2;2,1|1,-1> = -sqrt(5)/5
<2,-1;2,0|1,-1> = sqrt(30)/10
<2,0;2,-1|1,-1> = -sqrt(30)/10
<2,1;2,-2|1,-1> = sqrt(5)/5
<2,0;2,2|2,2> = sqrt(14)/7
<2,1;2,1|2,2> = -sqrt(21)/7
<2,2;2,0|2,2> = sqrt(14)/7
<2,-1;2,2|2,1> = sqrt(21)/7
<2,0;2,1|2,1> = -sqrt(14)/14
<2,1;2,0|2,1> = -sqrt(14)/14
<2,2;2,-1|2,1> = sqrt(21)/7
<2,-2;2,2|2,0> = sqrt(14)/7
<2,-1;2,1|2,0> = sqrt(14)/14
<2,0;2,0|2,0> = -sqrt(14)/7
<2,1;2,-1|2,0> = sqrt(14)/14
<2,2;2,-2|2,0> = sqrt(14)/7
<2,-2;2,1|2,-1> = sqrt(21)/7
<2,-1;2,0|2,-1> = -sqrt(14)/14
<2,0;2,-1|2,-1> = -sqrt(14)/14
<2,1;2,-2|2,-1> = sqrt(21)/7
<2,-2;2,0|2,-2> = sqrt(14)/7
<2,-1;2,-1|2,-2> = -sqrt(21)/7
